In [10]:
from api_keys import *

import os
import numpy as np
import pandas as pd
import requests 
from geopy.geocoders import Nominatim

In [11]:
def get_tiles(bounding_box, tiles):
    lat_max = bounding_box[3]
    lon_max = bounding_box[2]
    lat_min = bounding_box[1]
    lon_min = bounding_box[0]
    
    lat_range = lat_max - lat_min
    lon_range = lon_max - lon_min
    lat_step = lat_range / np.sqrt(tiles)
    lon_step = lon_range / np.sqrt(tiles)

    tile_coords = list()

    for i in range(tiles):
        for j in range(tiles):
            tile_lat_min = lat_min + i*lat_step
            tile_lon_min = lon_min + j*lon_step
            tile_lat_max = tile_lat_min + lat_step
            tile_lon_max = tile_lon_min + lon_step

            tile_coords.append([tile_lon_min, tile_lat_min, tile_lon_max, tile_lat_max])
    return tile_coords

In [16]:
def get_image(tile_coords):
    with open(f"data/tile_coords.csv", 'w') as coord_file:
        coord_file.write('image_name, min_lon, min_lat, max_lon, max_lat\n')

    for idx, bb in enumerate(tile_coords):
        api_url = f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/{bb}/1024x1024?access_token={MAPBOX_KEY}"
        response = requests.get(api_url)

        if response.status_code == 200:
            with open(f"beta_area/img_{idx:02.0f}.jpeg", "wb") as file:
                file.write(response.content)
            with open(f"beta_area/tile_coords.csv", 'a') as coord_file:
                coord_file.write(f"img_{idx:02.0f}, {bb[0]},{bb[1]},{bb[2]},{bb[3]}\n")
        else:
            print(f"Failed to save tile {idx}")

In [17]:
min_lon = -121.806611 #121°48'23.8"W
min_lat = 37.231278 #37°13'52.6"N
max_lon = -121.804028 #121°48'14.5"W
max_lat = 37.232833 #37°13'58.2"N

beta_area = [min_lon, min_lat, max_lon, max_lat]

In [18]:
coords = get_tiles(beta_area, 10)

In [21]:
coords[:5]

[[-121.806611, 37.231278, -121.80579418368038, 37.23176973417616],
 [-121.80579418368038, 37.231278, -121.80497736736075, 37.23176973417616],
 [-121.80497736736076, 37.231278, -121.80416055104114, 37.23176973417616],
 [-121.80416055104114, 37.231278, -121.80334373472151, 37.23176973417616],
 [-121.80334373472152, 37.231278, -121.8025269184019, 37.23176973417616]]

In [19]:
get_image(coords)